In [ ]:
%reload_ext autoreload
%autoreload 2
# %matplotlib widget

import os
import sys
sys.path.append(os.path.join('..'))
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
from src import utils
from src import visualizations as vis

from matplotlib.widgets import Slider
import matplotlib.animation as animation
from matplotlib import colormaps


os.makedirs('../outputs', exist_ok=True)

Initialize the spins in z direction

In [ ]:
Nrf = 100 # points
TB = 4 # Time x Bandwidth

z = np.arange(-1,1,0.05)
M = np.array([z*0, z*0, np.ones(len(z))]).T

Create a RF-Pulse

In [ ]:
rf = utils.msinc(Nrf, TB / 4)
rf = rf * 90/ np.sum(rf)

In [ ]:
fig, axs = plt.subplots(1, figsize=(7,3))
axs.plot(rf, linewidth=4)
axs.set_title("RF Pulse", fontsize=18)
axs.set_xlabel("Time", fontsize=16)
axs.set_ylabel("Amplitude", fontsize=16)

Initialize different phase shifts depending on z position of the spin

In [ ]:
phase = np.exp(np.pi*1j*z*TB*3/Nrf)
phmult = np.array([phase, np.conj(phase), np.ones(len(phase))]).T

In [ ]:
Mc = utils.mr2mc(M)

# transformation from real to complex
T = np.array([[1, 1j, 0], [1, -1j, 0], [0, 0, 1]])

# transformation to Mxy
Txy = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]])

In [ ]:
Mrs = []
for k in range(Nrf):
    R = T @ utils.rot_y(rf[k]) @ np.linalg.inv(T) # rotation in complex domain
    
    # rotate the magnetization by the amplitude of the RF pulse
    # apply the phase indeuced by the gradient field of the slice
    Mc = phmult * (R @ Mc.T).T 
    
    Mxy = (Txy @ utils.mc2mr(Mc).T).T

    # Mrs.append(Mxy)
    # or for z component
    Mrs.append(utils.mc2mr(Mc))

#### Gif view

In [ ]:

num_frames = len(Mrs)
# Set up figure
fig, axs = plt.subplots(1, 2, figsize=(8, 4))


# Animation function
def update(frame):
    vis.plotM_animated(axs, Mrs[frame], z)
    fig.tight_layout()
    fig.suptitle(f"Bloch Simulation: Slice Selection (frame {frame+1}/{num_frames})")
    

# Create animation
ani = animation.FuncAnimation(fig, update, frames=num_frames, repeat=False)

# Save as GIF
ani.save("../outputs/magnetic_moments.gif", writer="pillow", fps=10)
# # Save as Mp4
# ani.save("../outputs/magnetic_moments.mp4", writer="ffmpeg", fps=10)

#### Slider view

In [ ]:
num_frames = len(Mrs)
# Set up figure
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
ax_slider = plt.axes([0.2, 0.02, 0.6, 0.03], facecolor='lightgoldenrodyellow')
slider = Slider(ax_slider, 'Frame', 0, 99, valinit=0, valstep=1)

# Animation function
def update(frame):
    frame = slider.val
    vis.plotM_animated(axs, Mrs[frame], z)
    fig.suptitle(f"Frame {frame+1}/{num_frames}")

# Connect the slider to the update function
slider.on_changed(update)
plt.show()